In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils.class_weight import compute_class_weight
from nilearn.connectome import ConnectivityMeasure
import joblib

# Предобработка

In [34]:
def read_data(src=r'../data/ts_cut/ihb.npy'):
    data = np.load(src)
    return data

def defoult_preprocess(data):
    data = np.nan_to_num(data, nan=np.nanmean(data, axis=0))
    scaler = StandardScaler()
    for i in range(data.shape[0]):
        data[i] = scaler.fit_transform(data[i])
    return data

## Функции для обработки данных

In [35]:
def get_connectome(timeseries: np.ndarray,
                   conn_type: str = 'corr') -> np.ndarray:
    if conn_type == 'corr':
        conn = ConnectivityMeasure(kind='correlation', standardize=False).fit_transform(timeseries)
        conn[conn == 1] = 0.999999

        for i in conn:
            np.fill_diagonal(i, 0)

        conn = np.arctanh(conn)

    else:
        raise NotImplementedError

    return conn

def extract_upper_triangular(matrix):
    return matrix[np.triu_indices_from(matrix, k=1)]

In [ ]:
def corr_line(data):
    data_connectome = get_connectome(data)
    return np.array([extract_upper_triangular(mat) for mat in data_connectome])

### Matrix Profiles

In [ ]:
!pip install --user stumpy
!pip install --upgrade numpy==1.23.5
!pip install --upgrade numba

In [224]:
def matrix_Profiles(data, subseq_len=50):
    cluster_features = []
    for i in range(data.shape[0]):
        ts = data[i]  # Временной ряд (многомерный)
        mp = stumpy.mstump(ts, m=subseq_len)  # Расчет Matrix Profile для многомерного временного ряда
        # Усреднение по всем измерениям (можно попробовать другие способы агрегации)
        mean_mp = np.mean(mp, axis=0)
        cluster_features.append(mean_mp)
    return np.array(cluster_features)

### Symbolic Aggregate approXimation (SAX)

In [63]:
!pip install tslearn

  Obtaining dependency information for tslearn from https://files.pythonhosted.org/packages/97/22/8dba9a7149d51fe0b6163a5a6b7efc315ab3c097cb6b0d1fc649a03f2722/tslearn-0.6.3-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/374.4 kB ? eta -:--:--
   ---------------------------------------- 0.0/374.4 kB ? eta -:--:--
   --- ----------------------------------- 30.7/374.4 kB 660.6 kB/s eta 0:00:01
   ---------- --------------------------- 102.4/374.4 kB 991.0 kB/s eta 0:00:01
   --------------------------- ------------ 256.0/374.4 kB 1.8 MB/s eta 0:00:01
   ---------------------------------------  368.6/374.4 kB 2.1 MB/s eta 0:00:01
   ---------------------------------------- 374.4/374.4 kB 1.9 MB/s eta 0:00:00


In [70]:
from tslearn.piecewise import SymbolicAggregateApproximation


def apply_sax_to_data(data, n_segments=30, alphabet_size=7):
    if len(data.shape) != 3:
        raise ValueError("Data должно иметь размер (samples, timesteps, features)")
    
    # Инициализация SAX
    sax = SymbolicAggregateApproximation(n_segments=n_segments, alphabet_size_avg=alphabet_size)
    
    # Преобразуем каждый временной ряд для каждого признака
    sax_data = np.empty((data.shape[0], n_segments, data.shape[2]))
    
    for i in range(data.shape[2]):  # Применяем SAX к каждому признаку отдельно
        ts = data[:, :, i]  # Выбираем временной ряд для признака i
        sax_transformed = sax.fit_transform(ts)
        sax_data[:, :, i] = sax_transformed.squeeze()
    
    return sax_data

data = read_data()
data = defoult_preprocess(data)
data_sax = apply_sax_to_data(data)
print(data_sax.shape)

C:\Users\Public\anaconda\Lib\site-packages\tslearn\utils\utils.py:90: UserWarning: 2-Dimensional data passed. Assuming these are 320 1-dimensional timeseries
  warnings.warn(
C:\Users\Public\anaconda\Lib\site-packages\tslearn\piecewise\piecewise.py:152: RuntimeWarning: Mean of empty slice.
  X_transformed[i_ts, i_seg, :] = segment.mean(axis=0)
C:\Users\Public\anaconda\Lib\site-packages\numpy\core\_methods.py:121: RuntimeWarning: invalid value encountered in divide
  DeprecationWarning,
C:\Users\Public\anaconda\Lib\site-packages\tslearn\utils\utils.py:90: UserWarning: 2-Dimensional data passed. Assuming these are 320 1-dimensional timeseries
  warnings.warn(
C:\Users\Public\anaconda\Lib\site-packages\tslearn\piecewise\piecewise.py:152: RuntimeWarning: Mean of empty slice.
  X_transformed[i_ts, i_seg, :] = segment.mean(axis=0)
C:\Users\Public\anaconda\Lib\site-packages\numpy\core\_methods.py:121: RuntimeWarning: invalid value encountered in divide
  DeprecationWarning,
C:\Users\Public\an

C:\Users\Public\anaconda\Lib\site-packages\tslearn\utils\utils.py:90: UserWarning: 2-Dimensional data passed. Assuming these are 320 1-dimensional timeseries
  warnings.warn(
C:\Users\Public\anaconda\Lib\site-packages\tslearn\piecewise\piecewise.py:152: RuntimeWarning: Mean of empty slice.
  X_transformed[i_ts, i_seg, :] = segment.mean(axis=0)
C:\Users\Public\anaconda\Lib\site-packages\numpy\core\_methods.py:121: RuntimeWarning: invalid value encountered in divide
  DeprecationWarning,
C:\Users\Public\anaconda\Lib\site-packages\tslearn\utils\utils.py:90: UserWarning: 2-Dimensional data passed. Assuming these are 320 1-dimensional timeseries
  warnings.warn(
C:\Users\Public\anaconda\Lib\site-packages\tslearn\piecewise\piecewise.py:152: RuntimeWarning: Mean of empty slice.
  X_transformed[i_ts, i_seg, :] = segment.mean(axis=0)
C:\Users\Public\anaconda\Lib\site-packages\numpy\core\_methods.py:121: RuntimeWarning: invalid value encountered in divide
  DeprecationWarning,
C:\Users\Public\an

C:\Users\Public\anaconda\Lib\site-packages\tslearn\utils\utils.py:90: UserWarning: 2-Dimensional data passed. Assuming these are 320 1-dimensional timeseries
  warnings.warn(
C:\Users\Public\anaconda\Lib\site-packages\tslearn\piecewise\piecewise.py:152: RuntimeWarning: Mean of empty slice.
  X_transformed[i_ts, i_seg, :] = segment.mean(axis=0)
C:\Users\Public\anaconda\Lib\site-packages\numpy\core\_methods.py:121: RuntimeWarning: invalid value encountered in divide
  DeprecationWarning,
C:\Users\Public\anaconda\Lib\site-packages\tslearn\utils\utils.py:90: UserWarning: 2-Dimensional data passed. Assuming these are 320 1-dimensional timeseries
  warnings.warn(
C:\Users\Public\anaconda\Lib\site-packages\tslearn\piecewise\piecewise.py:152: RuntimeWarning: Mean of empty slice.
  X_transformed[i_ts, i_seg, :] = segment.mean(axis=0)
C:\Users\Public\anaconda\Lib\site-packages\numpy\core\_methods.py:121: RuntimeWarning: invalid value encountered in divide
  DeprecationWarning,
C:\Users\Public\an

C:\Users\Public\anaconda\Lib\site-packages\tslearn\utils\utils.py:90: UserWarning: 2-Dimensional data passed. Assuming these are 320 1-dimensional timeseries
  warnings.warn(
C:\Users\Public\anaconda\Lib\site-packages\tslearn\piecewise\piecewise.py:152: RuntimeWarning: Mean of empty slice.
  X_transformed[i_ts, i_seg, :] = segment.mean(axis=0)
C:\Users\Public\anaconda\Lib\site-packages\numpy\core\_methods.py:121: RuntimeWarning: invalid value encountered in divide
  DeprecationWarning,
C:\Users\Public\anaconda\Lib\site-packages\tslearn\utils\utils.py:90: UserWarning: 2-Dimensional data passed. Assuming these are 320 1-dimensional timeseries
  warnings.warn(
C:\Users\Public\anaconda\Lib\site-packages\tslearn\piecewise\piecewise.py:152: RuntimeWarning: Mean of empty slice.
  X_transformed[i_ts, i_seg, :] = segment.mean(axis=0)
C:\Users\Public\anaconda\Lib\site-packages\numpy\core\_methods.py:121: RuntimeWarning: invalid value encountered in divide
  DeprecationWarning,
C:\Users\Public\an

C:\Users\Public\anaconda\Lib\site-packages\tslearn\utils\utils.py:90: UserWarning: 2-Dimensional data passed. Assuming these are 320 1-dimensional timeseries
  warnings.warn(
C:\Users\Public\anaconda\Lib\site-packages\tslearn\piecewise\piecewise.py:152: RuntimeWarning: Mean of empty slice.
  X_transformed[i_ts, i_seg, :] = segment.mean(axis=0)
C:\Users\Public\anaconda\Lib\site-packages\numpy\core\_methods.py:121: RuntimeWarning: invalid value encountered in divide
  DeprecationWarning,
C:\Users\Public\anaconda\Lib\site-packages\tslearn\utils\utils.py:90: UserWarning: 2-Dimensional data passed. Assuming these are 320 1-dimensional timeseries
  warnings.warn(
C:\Users\Public\anaconda\Lib\site-packages\tslearn\piecewise\piecewise.py:152: RuntimeWarning: Mean of empty slice.
  X_transformed[i_ts, i_seg, :] = segment.mean(axis=0)
C:\Users\Public\anaconda\Lib\site-packages\numpy\core\_methods.py:121: RuntimeWarning: invalid value encountered in divide
  DeprecationWarning,
C:\Users\Public\an

...

C:\Users\Public\anaconda\Lib\site-packages\tslearn\utils\utils.py:90: UserWarning: 2-Dimensional data passed. Assuming these are 320 1-dimensional timeseries
  warnings.warn(
C:\Users\Public\anaconda\Lib\site-packages\tslearn\piecewise\piecewise.py:152: RuntimeWarning: Mean of empty slice.
  X_transformed[i_ts, i_seg, :] = segment.mean(axis=0)
C:\Users\Public\anaconda\Lib\site-packages\numpy\core\_methods.py:121: RuntimeWarning: invalid value encountered in divide
  DeprecationWarning,
C:\Users\Public\anaconda\Lib\site-packages\tslearn\utils\utils.py:90: UserWarning: 2-Dimensional data passed. Assuming these are 320 1-dimensional timeseries
  warnings.warn(
C:\Users\Public\anaconda\Lib\site-packages\tslearn\piecewise\piecewise.py:152: RuntimeWarning: Mean of empty slice.
  X_transformed[i_ts, i_seg, :] = segment.mean(axis=0)
C:\Users\Public\anaconda\Lib\site-packages\numpy\core\_methods.py:121: RuntimeWarning: invalid value encountered in divide
  DeprecationWarning,
C:\Users\Public\an

C:\Users\Public\anaconda\Lib\site-packages\tslearn\utils\utils.py:90: UserWarning: 2-Dimensional data passed. Assuming these are 320 1-dimensional timeseries
  warnings.warn(
C:\Users\Public\anaconda\Lib\site-packages\tslearn\piecewise\piecewise.py:152: RuntimeWarning: Mean of empty slice.
  X_transformed[i_ts, i_seg, :] = segment.mean(axis=0)
C:\Users\Public\anaconda\Lib\site-packages\numpy\core\_methods.py:121: RuntimeWarning: invalid value encountered in divide
  DeprecationWarning,
C:\Users\Public\anaconda\Lib\site-packages\tslearn\utils\utils.py:90: UserWarning: 2-Dimensional data passed. Assuming these are 320 1-dimensional timeseries
  warnings.warn(
C:\Users\Public\anaconda\Lib\site-packages\tslearn\piecewise\piecewise.py:152: RuntimeWarning: Mean of empty slice.
  X_transformed[i_ts, i_seg, :] = segment.mean(axis=0)
C:\Users\Public\anaconda\Lib\site-packages\numpy\core\_methods.py:121: RuntimeWarning: invalid value encountered in divide
  DeprecationWarning,
C:\Users\Public\an

C:\Users\Public\anaconda\Lib\site-packages\tslearn\utils\utils.py:90: UserWarning: 2-Dimensional data passed. Assuming these are 320 1-dimensional timeseries
  warnings.warn(
C:\Users\Public\anaconda\Lib\site-packages\tslearn\piecewise\piecewise.py:152: RuntimeWarning: Mean of empty slice.
  X_transformed[i_ts, i_seg, :] = segment.mean(axis=0)
C:\Users\Public\anaconda\Lib\site-packages\numpy\core\_methods.py:121: RuntimeWarning: invalid value encountered in divide
  DeprecationWarning,
C:\Users\Public\anaconda\Lib\site-packages\tslearn\utils\utils.py:90: UserWarning: 2-Dimensional data passed. Assuming these are 320 1-dimensional timeseries
  warnings.warn(
C:\Users\Public\anaconda\Lib\site-packages\tslearn\piecewise\piecewise.py:152: RuntimeWarning: Mean of empty slice.
  X_transformed[i_ts, i_seg, :] = segment.mean(axis=0)
C:\Users\Public\anaconda\Lib\site-packages\numpy\core\_methods.py:121: RuntimeWarning: invalid value encountered in divide
  DeprecationWarning,
C:\Users\Public\an

C:\Users\Public\anaconda\Lib\site-packages\tslearn\utils\utils.py:90: UserWarning: 2-Dimensional data passed. Assuming these are 320 1-dimensional timeseries
  warnings.warn(
C:\Users\Public\anaconda\Lib\site-packages\tslearn\piecewise\piecewise.py:152: RuntimeWarning: Mean of empty slice.
  X_transformed[i_ts, i_seg, :] = segment.mean(axis=0)
C:\Users\Public\anaconda\Lib\site-packages\numpy\core\_methods.py:121: RuntimeWarning: invalid value encountered in divide
  DeprecationWarning,
C:\Users\Public\anaconda\Lib\site-packages\tslearn\utils\utils.py:90: UserWarning: 2-Dimensional data passed. Assuming these are 320 1-dimensional timeseries
  warnings.warn(
C:\Users\Public\anaconda\Lib\site-packages\tslearn\piecewise\piecewise.py:152: RuntimeWarning: Mean of empty slice.
  X_transformed[i_ts, i_seg, :] = segment.mean(axis=0)
C:\Users\Public\anaconda\Lib\site-packages\numpy\core\_methods.py:121: RuntimeWarning: invalid value encountered in divide
  DeprecationWarning,
C:\Users\Public\an

(320, 30, 246)


In [328]:
data_sax

array([[[6., 6., 6., ..., 6., 6., 6.],
        [6., 6., 6., ..., 6., 6., 6.],
        [6., 6., 6., ..., 6., 6., 6.],
        ...,
        [6., 6., 6., ..., 6., 6., 6.],
        [6., 6., 6., ..., 6., 6., 6.],
        [6., 6., 6., ..., 6., 6., 6.]],

       [[6., 6., 6., ..., 6., 6., 6.],
        [6., 6., 6., ..., 6., 6., 6.],
        [6., 6., 6., ..., 6., 6., 6.],
        ...,
        [6., 6., 6., ..., 6., 6., 6.],
        [6., 6., 6., ..., 6., 6., 6.],
        [6., 6., 6., ..., 6., 6., 6.]],

       [[6., 6., 6., ..., 6., 6., 6.],
        [6., 6., 6., ..., 6., 6., 6.],
        [6., 6., 6., ..., 6., 6., 6.],
        ...,
        [6., 6., 6., ..., 6., 6., 6.],
        [6., 6., 6., ..., 6., 6., 6.],
        [6., 6., 6., ..., 6., 6., 6.]],

       ...,

       [[6., 6., 6., ..., 6., 6., 6.],
        [6., 6., 6., ..., 6., 6., 6.],
        [6., 6., 6., ..., 6., 6., 6.],
        ...,
        [6., 6., 6., ..., 6., 6., 6.],
        [6., 6., 6., ..., 6., 6., 6.],
        [6., 6., 6., ..., 6., 6.

# Кластеризация

In [55]:
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering

In [56]:
n_clusters = 20

In [230]:
data = read_data()
data = defoult_preprocess(data)
print(data.shape)
data = matrix_Profiles(data, 130)
data = corr_line(data)
data.shape

(320, 10, 246)


(320, 6786)

In [287]:
# K-means
kmeans = KMeans(n_clusters=20)
labels = kmeans.fit_predict(cluster_features)

# Agglomerative Clustering
# agg_clustering = AgglomerativeClustering(n_clusters=n_clusters, linkage='ward')
# labels = agg_clustering.fit_predict(data)

df_labels = pd.DataFrame({'prediction': labels}).apply(lambda row: row.value_counts(normalize=True), axis=1).fillna(0).sum(axis=0)
((df_labels - 16)**2).sum()

3346.0

In [289]:
df_labels

0      5.0
1     23.0
2     25.0
3     19.0
4      8.0
5     16.0
6      5.0
7      3.0
8     26.0
9     10.0
10    17.0
11    60.0
12    31.0
13     9.0
14    18.0
15     2.0
16    14.0
17     7.0
18    16.0
19     6.0
dtype: float64

### Свой KMeans

In [344]:
read_data().shape

(320, 10, 246)

In [333]:
import numpy as np
from sklearn.metrics import pairwise_distances_argmin_min

class ConstrainedKMeans:
    def __init__(self, n_clusters, max_elements_per_cluster, max_iter=300, tol=1e-4):
        self.n_clusters = n_clusters
        self.max_elements_per_cluster = max_elements_per_cluster
        self.max_iter = max_iter
        self.tol = tol

    def fit(self, X):
        n_samples, n_features = X.shape
        self.centers = X[np.random.choice(n_samples, self.n_clusters, replace=False)]
        
        labels = np.zeros(n_samples, dtype=int)
        
        for iteration in range(self.max_iter):
            # Присвоение кластеров
            new_labels, _ = pairwise_distances_argmin_min(X, self.centers)

            # Подсчет количества элементов в каждом кластере
            cluster_counts = np.bincount(new_labels, minlength=self.n_clusters)

            # Проверка на максимальное количество элементов в каждом кластере
            for cluster_id in range(self.n_clusters):
                if cluster_counts[cluster_id] > self.max_elements_per_cluster:
                    # Получаем индексы элементов, принадлежащих текущему кластеру
                    idx_to_keep = np.where(new_labels == cluster_id)[0]
                    
                    # Случайным образом выбираем нужное количество элементов
                    if len(idx_to_keep) > self.max_elements_per_cluster:
                        idx_to_remove = np.random.choice(idx_to_keep, cluster_counts[cluster_id] - self.max_elements_per_cluster, replace=False)
                        new_labels[idx_to_remove] = -1  # Убираем элементы из этого кластера

            # Переопределение меток для неподписанных экземпляров
            unassigned_indices = np.where(new_labels == -1)[0]
            if len(unassigned_indices) > 0:
                nearest_clusters = self.predict(X[unassigned_indices])
                
                for idx, nearest in zip(unassigned_indices, nearest_clusters):
                    if cluster_counts[nearest] < self.max_elements_per_cluster:
                        new_labels[idx] = nearest
                    else:
                        # Принудительное назначение в кластер с минимальным количеством элементов
                        least_filled_cluster = np.argmin(cluster_counts)
                        new_labels[idx] = least_filled_cluster

            # Обновление меток
            labels = new_labels

            # Пересчет центров кластеров
            new_centers = np.array([X[labels == i].mean(axis=0) if np.any(labels == i) else self.centers[i] for i in range(self.n_clusters)])

            # Проверка на сходимость
            if np.linalg.norm(new_centers - self.centers) < self.tol:
                break

            self.centers = new_centers

        # Финальная проверка меток
        # Если все экземпляры не распределены, принудительно назначаем им метки
        self.labels_ = labels
        unique_labels = np.unique(self.labels_)
        
        if len(unique_labels) > self.n_clusters:
            # Удаляем лишние метки
            label_map = {}
            new_label_count = 0
            
            for i in range(len(self.labels_)):
                if self.labels_[i] not in label_map:
                    if new_label_count < self.n_clusters:
                        label_map[self.labels_[i]] = new_label_count
                        new_label_count += 1
                    else:
                        # Если превышено количество меток, назначаем последним меткам
                        label_map[self.labels_[i]] = new_label_count - 1

            # Применяем новую нумерацию меток
            self.labels_ = np.array([label_map[label] for label in self.labels_])

        return self

    def predict(self, X):
        return pairwise_distances_argmin(X, self.centers)

In [311]:
data = defoult_preprocess(read_data())
data = matrix_Profiles(data, 130)

In [314]:
data = corr_line(data)

In [315]:
kmeans = ConstrainedKMeans(n_clusters=20, max_elements_per_cluster=16)
kmeans.fit(data)
kmeans.labels_

array([ 1, 16,  6, 14, 16, 12,  9,  2,  5, 12,  0,  6,  6, 10,  2,  4, 18,
       17, 17, 12, 12,  3, 12,  3,  9,  4, 13,  4,  5,  6,  7, 10,  9, 19,
        1,  4, 19, 12, 11, 11,  4,  1,  1, 11, 14,  6,  3, 17,  9, 11,  3,
        5, 16,  8, 15,  1,  1, 11,  2, 15,  2, 15,  9,  3,  0, 18,  5,  7,
        1, 19,  7, 18, 13, 10,  7, 18,  2, 15,  2, 12, 12, 10,  1,  7, 19,
       14,  8, 13, 11,  9, 18,  2,  1, 15, 14, 10, 10, 11,  0, 16,  3, 10,
        0, 19, 16, 19, 13, 11, 19,  2, 17, 19, 14,  6,  4,  4,  5, 14, 16,
       12, 16, 19, 14,  6, 16,  5, 16,  8,  9, 14, 13, 14,  4, 17, 12,  7,
        5,  3,  5, 11, 19,  2, 14, 19, 11, 12, 18,  7, 18, 15, 17, 13,  1,
        2, 10, 15,  6, 15,  4, 10,  6, 13,  1, 11,  4,  7, 16, 18, 17, 13,
       15,  7, 16,  4, 15,  4,  2,  7, 19, 18,  5,  3, 14, 13, 15, 10,  3,
       17, 16,  4, 10, 19, 12, 19,  8,  1,  8, 18, 13,  6,  5,  0,  4, 12,
        3,  9, 15, 11,  5, 17,  2, 11, 11,  8, 17, 19,  0,  0,  6,  5,  8,
       14, 17, 16,  5,  8

In [316]:
df_labels = pd.DataFrame({'prediction': kmeans.labels_}).apply(lambda row: row.value_counts(normalize=True), axis=1).fillna(0).sum(axis=0)
((df_labels - 16)**2).sum()

56.0

In [317]:
df_labels

0     15.0
1     15.0
2     16.0
3     15.0
4     16.0
5     16.0
6     16.0
7     15.0
8     15.0
9     16.0
10    16.0
11    16.0
12    15.0
13    16.0
14    16.0
15    16.0
16    16.0
17    16.0
18    15.0
19    23.0
dtype: float64

### Cross-Correlation Clustering

In [125]:
import numpy as np
from sklearn.cluster import KMeans
from scipy.signal import correlate

In [126]:
def compute_cross_correlation(data):
    num_samples = data.shape[0]
    correlation_matrix = np.zeros((num_samples, num_samples))

    for i in range(num_samples):
        for j in range(num_samples):
            correlation = correlate(data[i], data[j], mode='valid')
            correlation_matrix[i, j] = np.max(correlation)  # Сохраняем максимальную корреляцию

    return correlation_matrix

In [201]:
data.shape

(320, 10, 246)

In [329]:
data = defoult_preprocess(read_data())
data = corr_line(data)
data = compute_cross_correlation(data)
data.shape

(320, 320)

In [337]:
kmeans = ConstrainedKMeans(n_clusters=20, max_elements_per_cluster=16)
kmeans.fit(data)
kmeans.labels_

array([16,  3,  2,  2,  2,  3,  2, 16, 18,  2,  2,  2,  2,  2,  8, 12, 17,
        2, 19,  2,  2,  9, 19, 16, 11,  1,  2,  7,  2, 16,  2, 15, 16,  2,
        2,  2,  2,  7,  2,  2,  2,  2,  9,  3,  2,  2,  7,  2,  2,  2, 16,
       19,  2,  2,  3, 18,  2,  2, 12, 17, 13,  1,  2, 10,  2,  2,  3,  2,
       15,  2, 12, 15,  9,  2,  2,  3, 16,  2,  9,  3,  9, 19,  2,  2,  1,
        2, 12,  2,  2,  2, 13, 16,  3, 19,  5,  7, 15,  2, 19, 19,  2,  2,
        4, 19,  2,  2, 17, 12,  2,  2, 13,  2,  2,  7,  2,  2,  2,  2,  8,
       19,  7,  2,  2,  1, 12,  2,  2,  8,  2,  0,  2,  2, 19, 19,  2,  1,
       13,  7,  1,  2,  2,  2,  7,  3,  2, 18,  2, 12,  9,  2, 19,  2,  2,
       19,  5,  2,  9, 13,  2, 13, 12, 12,  2,  2,  2,  2,  2,  7,  2, 12,
        2,  2,  2,  2, 16,  7,  2,  2,  2,  2, 12, 18,  2,  2,  2,  2,  2,
        0, 16,  2,  2,  2,  2,  1,  3,  2,  9,  2,  2,  3,  8,  3,  3,  2,
       14,  2, 16,  2,  5,  2,  2,  4, 12, 16,  2, 16,  2,  2,  2,  2,  2,
        0, 12,  2, 19,  2

In [338]:
df_labels = pd.DataFrame({'prediction': kmeans.labels_}).apply(lambda row: row.value_counts(normalize=True), axis=1).fillna(0).sum(axis=0)
((df_labels - 16)**2).sum()

24292.0

In [339]:
df_labels

0       4.0
1       8.0
2     166.0
3      16.0
4       4.0
5       4.0
6       2.0
7      16.0
8       6.0
9      16.0
10      2.0
11      2.0
12     16.0
13      9.0
14      2.0
15      5.0
16     16.0
17      5.0
18      5.0
19     16.0
dtype: float64

In [340]:
pd.DataFrame({'prediction': labels}).apply(lambda row: row.value_counts(normalize=True), axis=1).fillna(0).sum(axis=0)

0      5.0
1     23.0
2     25.0
3     19.0
4      8.0
5     16.0
6      5.0
7      3.0
8     26.0
9     10.0
10    17.0
11    60.0
12    31.0
13     9.0
14    18.0
15     2.0
16    14.0
17     7.0
18    16.0
19     6.0
dtype: float64

### Deep Embedded Clustering (DEC)

In [104]:
import numpy as np
from tensorflow import keras
from sklearn.cluster import KMeans

In [120]:
data = defoult_preprocess(read_data())
data = corr_line(data)

input_dim = data.shape[1]  # 30135, размерность данных
encoding_dim = 128  # Размерность скрытого слоя

# Создание автокодировщика
autoencoder = keras.Sequential([
    keras.layers.Input(shape=(input_dim,)),
    keras.layers.Dense(encoding_dim, activation='relu'),
    keras.layers.Dense(input_dim, activation='sigmoid')
])

# Компиляция модели
autoencoder.compile(optimizer='adam', loss='mse')

# Обучение модели
autoencoder.fit(data, data, epochs=10, batch_size=256, shuffle=True)

# Теперь создаем кодировщик
encoder = keras.Model(inputs=autoencoder.input, outputs=autoencoder.layers[1].output)
features = encoder.predict(data)

# Кластеризация с помощью KMeans
kmeans = KMeans(n_clusters=20)  # Указываем количество кластеров
labels = kmeans.fit_predict(features)

print(labels)

Epoch 1/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 78ms/step - loss: 0.2672
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - loss: 0.2643
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - loss: 0.2401
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - loss: 0.1668
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - loss: 0.0749
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - loss: 0.0276
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - loss: 0.0185
Epoch 8/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - loss: 0.0173
Epoch 9/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - loss: 0.0172
Epoch 10/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - loss: 0.0172


AttributeError: The layer sequential_11 has never been called and thus has no defined input.

In [320]:
!pip install networkx 
!pip install python-louvain

  Using cached python-louvain-0.16.tar.gz (204 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [1 lines of output]
  ERROR: Can not execute `setup.py` since setuptools is not available in the build environment.
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

Encountered error while generating package metadata.

See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


### Time Series Segmentation + Clustering

In [343]:
import numpy as np
from sklearn.cluster import KMeans
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from tslearn.clustering import TimeSeriesKMeans

def segment_time_series(data, window_size):
    segments = []
    for time_series in data:
        for start in range(len(time_series) - window_size + 1):
            segments.append(time_series[start:start + window_size])
    return np.array(segments)

data = defoult_preprocess(read_data())
data = corr_line(data)

window_size = 5  # Размер окна для сегментации
segmented_data = segment_time_series(data, window_size)

# Нормализация данных
scaler = TimeSeriesScalerMeanVariance()
normalized_data = scaler.fit_transform(segmented_data)

# Кластеризация сегментов с помощью TimeSeriesKMeans
kmeans = TimeSeriesKMeans(n_clusters=10, metric="euclidean")
labels = kmeans.fit_predict(normalized_data)

print(labels)

KeyboardInterrupt: 

# Сохранение результатов

In [318]:
pd.DataFrame({'prediction': kmeans.labels_}).to_csv('../submission.csv', index=False)

In [196]:
pd.read_csv('../submission.csv').apply(lambda row: row.value_counts(normalize=True), axis=1).fillna(0).sum(axis=0)

0     18.0
1     20.0
2     10.0
3     12.0
4     34.0
5     28.0
6      6.0
7     18.0
8     24.0
9      8.0
10    28.0
11    14.0
12    16.0
13     6.0
14     8.0
15     8.0
16    10.0
17    30.0
18    12.0
19    10.0
dtype: float64